In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

import os
from os import PathLike
from pathlib import Path
import re

from aind_vr_foraging_analysis.utils.parsing import parse, AddExtraColumns
import aind_vr_foraging_analysis.utils.plotting as plotting
from aind_vr_foraging_analysis.utils import breathing_signal as breathing

# Plotting libraries
import matplotlib.pyplot as plt

import seaborn as sns
import pandas as pd
import numpy as np
import datetime

sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

color1='#d95f02'
color2='#1b9e77'
color3='#7570b3'
color4='yellow'
odor_list_color = [color1, color2, color3, color4]

pdf_path = r'Z:\stage\vr-foraging\sessions'
base_path = 'Z:/stage/vr-foraging/data/'
foraging_figures = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\results'

In [ ]:
class MetricsVrForaging():
    def __init__(self, session_path: PathLike):
        self.session_path = Path(session_path)
        self.data = parse.load_session_data(self.session_path)
        self.session = self.data['config'].streams.session_input.data['date'][:10]
        self.mouse = int(self.data['config'].streams.session_input.data['subject'])
        self.stage = self.data['config'].streams.tasklogic_input.data['stage_name']
        self.rig_name = self.data['config'].streams.rig_input.data['rig_name']

        print(self.rig_name)
        print(self.stage)
        if self.stage == 'thermistor screening':
            return
        
        self.data = parse.load_session_data(self.session_path)
        self.stage = self.data['config'].streams.tasklogic_input.data['stage_name']
        self.stream_data = parse.ContinuousData(self.data)
        self.total_epochs = parse.parse_dataframe(self.data)
        self.active_site = parse.AddExtraColumns(self.total_epochs, run_on_init=True).total_epochs
        self.reward_sites = self.active_site.loc[self.active_site['label'] == 'OdorSite']
        self.reward_sites['odor_sites'] = np.arange(1, len(self.reward_sites)+1)
        self.df = self.retrieve_metrics()

    def retrieve_metrics(self) -> pd.DataFrame:
        reward_sites = self.reward_sites
        active_site = self.active_site
        data = self.data

        df = pd.DataFrame()
        # Summary of different relevants aspects -------------------------------------------------

        unrewarded_stops = reward_sites.loc[reward_sites.is_reward==0]['reward_amount'].count()
        rewarded_stops = reward_sites.loc[reward_sites.is_reward==1]['reward_amount'].count()
        water_collected = reward_sites.loc[(reward_sites['is_reward']==1)]['reward_amount'].sum()
        total_stops = reward_sites.loc[(reward_sites['is_choice']==True)]['reward_amount'].count()

        print('Total sites: ' ,len(reward_sites), ' | ', 'Total rewarded stops: ',rewarded_stops, '(',  np.round((rewarded_stops/total_stops)*100,2),'%) | ', 
            'Total unrewarded stops: ',unrewarded_stops,'(',  np.round((unrewarded_stops/total_stops)*100,2),'%) | ','Water consumed: ', water_collected, 'ul')

        print('Total travelled m: ', np.round(active_site.start_position.max()/100,2), ', current position (cm): ', data['operation_control'].streams.CurrentPosition.data.max()[0]
        )

        for odor_label in reward_sites.odor_label.unique():
            values = reward_sites.loc[(reward_sites['odor_label']==odor_label)&(reward_sites['is_reward']==1)]['reward_amount'].sum()
            print(f'{odor_label} {values} ul')
            
        df.at[0,'odor_sites_travelled'] = int(len(reward_sites))
        df.at[0,'distance_m'] = data['operation_control'].streams.CurrentPosition.data.max()[0]/100
        df.at[0,'water_collected_ul'] = water_collected
        df.at[0,'rewarded_stops'] = int(rewarded_stops)
        df.at[0,'total_stops'] = int(total_stops)
        # df.at[0,'session_duration_min'] = (reward_sites.index[-1] - reward_sites.index[0])/60
        df.at[0, 'total_patches_visited'] = reward_sites.loc[reward_sites['site_number'] >= 1].patch_number.nunique()
        return df

    def retrieve_updater_values(self):
        # Initialize a pointer for the data values
        data_pointer = 0
        
        reward_sites = self.reward_sites
        data = self.data
        df = self.df
        
        # Save the updater values
        stop_duration = data['updater_events'].streams.UpdaterStopDurationOffset.data['data']
        stop_duration.reset_index(drop=True, inplace=True)
        delay = data['updater_events'].streams.UpdaterRewardDelayOffset.data['data']
        delay.reset_index(drop=True, inplace=True)
        velocity_threshold = data['updater_events'].streams.UpdaterStopVelocityThreshold.data['data']
        velocity_threshold.reset_index(drop=True, inplace=True)
        
        # Create a new column in reward_sites to store the updated values
        reward_sites['delay_s'] = None
        reward_sites['velocity_threshold_cms'] = None
        reward_sites['stop_duration_s'] = None

        try:
            # Iterate through each row of reward_sites
            for index, row in reward_sites.iterrows():
                if row['is_reward'] == 1:
                    # Copy the next available value from data and move the pointer
                    reward_sites.at[index, 'delay_s'] = delay[data_pointer]
                    reward_sites.at[index, 'velocity_threshold_cms'] = velocity_threshold[data_pointer]
                    reward_sites.at[index, 'stop_duration_s'] = stop_duration[data_pointer]
                    data_pointer += 1
                else:
                    # Copy the same value without moving the pointer
                    reward_sites.at[index, 'delay_s'] = delay[data_pointer]
                    reward_sites.at[index, 'velocity_threshold_cms'] = velocity_threshold[data_pointer]
                    reward_sites.at[index, 'stop_duration_s'] = stop_duration[data_pointer]
        except KeyError:
                reward_sites.at[index, 'delay_s'] = max(delay)
                reward_sites.at[index, 'velocity_threshold_cms'] = max(velocity_threshold)
                reward_sites.at[index, 'stop_duration_s'] = max(stop_duration)

        # Summary of the training metrics
        reward_sites['odor_sites'] = np.arange(1, len(reward_sites)+1)
        df.at[0,'start_delay'] = reward_sites['delay_s'].min()
        df.at[0,'end_delay'] = reward_sites['delay_s'].max()
        df.at[0, 'sites_to_max_delay'] = reward_sites[reward_sites['delay_s'] == reward_sites['delay_s'].max()].iloc[0]['odor_sites']
        df.at[0,'start_stop_duration'] = reward_sites['stop_duration_s'].min()
        df.at[0,'end_stop_duration'] = reward_sites['stop_duration_s'].max()
        df.at[0, 'sites_to_max_stop_duration'] = reward_sites[reward_sites['stop_duration_s'] == reward_sites['stop_duration_s'].max()].iloc[0]['odor_sites']
        df.at[0, 'rewarded_sites_in_max_stop'] = int(reward_sites[(reward_sites['stop_duration_s'] == reward_sites['stop_duration_s'].max())&(reward_sites.is_choice == 1)]['odor_sites'].nunique())

        df.at[0,'start_velocity_threshold'] = reward_sites['velocity_threshold_cms'].min()
        df.at[0,'end_velocity_threshold'] = reward_sites['velocity_threshold_cms'].max()
        df.at[0,'target_max_velocity_threshold'] = reward_sites['velocity_threshold_cms'].max()
        df.at[0, 'sites_to_min_velocity'] = reward_sites[reward_sites['velocity_threshold_cms'] == reward_sites['velocity_threshold_cms'].min()].iloc[0]['odor_sites']
            
        self.reward_sites = reward_sites
        self.df = df

    def get_metrics(self):
        return self.df

    def get_reward_sites(self):
        return self.reward_sites
    
    def get_mouse_and_session(self):
        return self.mouse, self.session
    
    def run_pdf_summary(self):
        color1='#d95f02'
        color2='#1b9e77'
        color3='#7570b3'
        color4='#e7298a'

        color_dict_label = {'Ethyl Butyrate': color1, 'Alpha-pinene': color1, 'Amyl Acetate': color3, 'Eugenol' : color3,
                            '2-Heptanone' : color2, 'Methyl Acetate': color1, 'Fenchone': color3, '2,3-Butanedione': color4, 'Methyl Butyrate': color2,}
        
        self.stream_data = parse.ContinuousData(self.data)
        encoder_data = self.stream_data.encoder_data
        odor_sites = AddExtraColumns(self.reward_sites, self.active_site, run_on_init=True).reward_sites
        active_site = AddExtraColumns(odor_sites, self.active_site).add_time_previous_intersite_interpatch()
        active_site['duration_epoch'] = active_site.index.to_series().diff().shift(-1)
        active_site['mouse'] = self.mouse
        active_site['session'] = self.session
        
        # Remove segments where the mouse was disengaged
        last_engaged_patch = odor_sites['patch_number'][odor_sites['skipped_count'] >= 10].min()
        if pd.isna(last_engaged_patch):
            last_engaged_patch = odor_sites['patch_number'].max()
            
        odor_sites['engaged'] = odor_sites['patch_number'] <= last_engaged_patch  
        
        # Recover color palette
        dict_odor = {}
        list_patches = parse.TaskSchemaProperties(self.data).patches
        for i, patches in enumerate(list_patches):
            # color_dict_label[patches['label']] = odor_list_color[i]
            dict_odor[i] = patches['label']
        
        trial_summary = plotting.trial_collection(odor_sites[['is_choice', 'site_number', 'odor_label', 'odor_sites', 'is_reward','depleted',
                                                                'reward_probability','reward_amount','reward_available']], 
                                                  encoder_data, 
                                                  self.mouse, 
                                                  self.session, 
                                                  window=(-1,3)
                                                )
    
        # Save each figure to a separate page in the PDF
        pdf_filename = f'{self.mouse}_{self.session}_summary.pdf'
            
        # return pdf_filename
    


In [ ]:
cum_session = pd.DataFrame(columns=['mouse', 'session', 'stage','simplified_stage'])
odor_sites_sum = pd.DataFrame()
cum_trial_summary = pd.DataFrame()

In [ ]:
mouse_list = ['828417', '828418', '828420', '828421', '828422', '828423', '828425', '815102', '815103', '815104', '795133', '808729', '807093', '807086', '806527']

date = datetime.date.today()
date_string = "09/21/2025"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()
        

# stage_progression = {'stageA_v1': 'Stage A',

In [ ]:
from aind_vr_foraging_analysis.utils.parsing import data_access
date_string = "2025-11-05" # YYYY-MM-DD

cum_df = pd.DataFrame()
velocity_cum = pd.DataFrame()

# fig, axes = plt.subplots(2,1, figsize=(12, 8))
mouse_list = ['810761', '811026', '828417', '815102', '795133', '806527', '815104', '798279']
for mouse in mouse_list:
    # This section will look at all the session paths that fulfill the condition
    session_paths = data_access.find_sessions_relative_to_date(
        mouse=mouse,
        date_string=date_string,
        when='on_or_after'
    )

    # session_paths  = [Path(rf'Z:\scratch\vr-foraging\PR test\828417_2025-11-07T182337Z')]

    # Iterate over the session paths and load the data
    for session_path in session_paths:
        # print(f"Loading {session_path.name}...")
        try:
            all_epochs, stream_data, data = data_access.load_session(
                session_path, extra=False
            )
            print(f"Loaded {session_path.name} successfully.")
        except Exception as e:
            print(f"Error loading {session_path.name}: {e}")
        
        all_epochs['mouse'] = mouse
        utc = pytz.UTC
        local_tz = pytz.timezone("America/Los_Angeles")

        utc_str = data['config'].streams.session_input.data['date']
        utc_dt = datetime.fromisoformat(utc_str)        # full timestamp
        local_dt = utc_dt.replace(tzinfo=utc).astimezone(local_tz)

        all_epochs['session'] = str(local_dt.strftime("%Y-%m-%d_%H"))  
        
        all_epochs['box'] = data['config'].streams.rig_input.data['rig_name']
        odor_sites = all_epochs.loc[all_epochs['label'] == 'OdorSite']
    
        print('\n'+file_name)
        parsed_session = MetricsVrForaging(session_path)
        if parsed_session.stage == 'thermistor screening':
            continue
        
        df = parsed_session.get_metrics()
        df['trainer'] = trainer_dict[mouse]
        df['session'] = parsed_session.session
        df['stage'] = parsed_session.stage
        df['rig'] = parsed_session.rig_name
        df['mouse'] = mouse 
        
        try:
            simplified_stage = re.search(r'stage([A-Za-z])', parsed_session.stage).group(1)
        except:
            simplified_stage = parsed_session.stage
            
        df['simplified_stage'] = simplified_stage
        
        reward_sites = parsed_session.get_reward_sites()

        if simplified_stage[:4] == 'stage' or simplified_stage == 'C' or parsed_session.stage[:7] == 'shaping':
            parsed_session.retrieve_updater_values()
            print(
            'Total sites travelled: ' + str(df.odor_sites_travelled.iloc[0]),
            '\nRewarded stops in max stop duration: ' + str(df.rewarded_sites_in_max_stop.iloc[0]),
            '\nTotal patches visited: ' + str(df.total_patches_visited.iloc[0]))
        
        reward_sites['mouse'] = mouse
        reward_sites['session'] = parsed_session.session
        reward_sites['stage'] = parsed_session.stage
        reward_sites['simplified_stage'] = simplified_stage
        reward_sites['rig'] = parsed_session.rig_name
        
        odor_sites_sum = pd.concat([odor_sites_sum, reward_sites], axis=0)
        
        if cum_session.loc[(cum_session.mouse == mouse)&(cum_session.session == parsed_session.session)].empty:
            df['session_n'] = len(cum_session.loc[(cum_session.mouse == mouse)]) + 1
            df['stage_session_n'] = len(cum_session.loc[(cum_session.mouse == mouse)&(cum_session.simplified_stage == simplified_stage)]) + 1
            df['m_min'] = df['distance_m']/30

            cum_session = pd.concat([cum_session, df], axis=0)
        
        stream_data = parsed_session.stream_data
        signal = stream_data.breathing.data.values
        zero_index = stream_data.breathing.index[0]

        # Define filter parameters
        cutoff_freq = 1  # cutoff frequency in Hz
        fs = 250  # sampling frequency in Hz

        # Apply high-pass filter
        filtered_breathing = breathing.highpass_filter(signal, cutoff_freq, fs)

        cutoff_freq = 40  # cutoff frequency in Hz
        filtered_breathing = breathing.lowpass_filter(filtered_breathing, cutoff_freq, fs)
        filtered_breathing = pd.DataFrame(filtered_breathing, index=stream_data.breathing.index-zero_index, columns=['breathing'])
        filtered_breathing['mouse'] = mouse
        filtered_breathing['rig'] = parsed_session.rig_name
        filtered_breathing.reset_index(inplace=True)
        cum_breathing = pd.concat([cum_breathing, filtered_breathing])
        
        trial_summary = plotting.trial_collection(reward_sites[['is_choice', 'site_number', 'odor_label', 'odor_sites', 'rig', 'is_reward',
                                                        'reward_probability','reward_amount','reward_available']], 
                                            stream_data.encoder_data, 
                                            mouse, 
                                            session, 
                                            window=(-1,3)
                                        )
        cum_trial_summary = pd.concat([cum_trial_summary, trial_summary], axis=0)

In [ ]:
max_range = cum_trial_summary.speed.max()
window = (-1, 3)
color_dict_label = {'Methyl Butyrate': color1, 'Alpha-pinene': color2, 'Amyl Acetate': color3, 'Eugenol' : color3}
mean=True   
n_odors = trial_summary.odor_label.unique()


for rig in cum_session.rig.unique():
    fig, ax1 = plt.subplots(
    1, len(n_odors), figsize=(len(n_odors) * 3.5, 5), sharex=True, sharey=True
)
    plt.suptitle(f"Rig {rig}")
    for j, odor_label in enumerate(n_odors):
        if len(n_odors) != 1:
            ax = ax1[j]
            ax1[0].set_ylabel("Velocity (cm/s)")
        else:
            ax = ax1
            ax.set_ylabel("Velocity (cm/s)")

        ax.set_xlabel("Time after odor onset (s)")
        ax.set_title(f"Patch {odor_label}")
        # ax.set_subtitle(f"Rig {rig}")
        ax.set_ylim(-13, max_range)
        ax.set_xlim(window)
        ax.hlines(
            5, window[0], window[1], color="black", linewidth=1, linestyles="dashed"
        )
        ax.fill_betweenx(
            np.arange(-20, max_range, 0.1),
            0,
            window[1],
            color=color_dict_label[odor_label],
            alpha=0.5,
            linewidth=0,
        )
        ax.fill_betweenx(
            np.arange(-20, max_range, 0.1),
            window[0],
            0,
            color="grey",
            alpha=0.3,
            linewidth=0,
        )

        df_results = (
            cum_trial_summary.loc[
                (cum_trial_summary.odor_label == odor_label)
                & (cum_trial_summary.is_choice == 0) 
                & (cum_trial_summary.rig == rig)
            ]
            .groupby(["odor_sites", "times", "odor_label", "mouse"])[["speed"]]
            .median()
            .reset_index()
        )

        if df_results.empty:
            continue

        if mean:
            sns.lineplot(
                x="times",
                y="speed",
                data=df_results,
                color="black",
                ci='sd',
                legend=False,
                linewidth=2,
                ax=ax,
                palette='tab20',
                hue='mouse'
            )

    sns.despine()
    plt.tight_layout()
    plt.show()

In [ ]:
max_range = cum_trial_summary.speed.max()
window = (-1, 3)
color_dict_label = {'Methyl Butyrate': color1, 'Alpha-pinene': color2, 'Amyl Acetate': color3, 'Eugenol' : color3}
mean=True   
n_odors = trial_summary.odor_label.unique()


for rig in cum_session.rig.unique():
    fig, ax1 = plt.subplots(
    1, len(n_odors), figsize=(len(n_odors) * 3.5, 5), sharex=True, sharey=True
)
    plt.suptitle(f"Rig {rig}")
    for j, odor_label in enumerate(n_odors):
        if len(n_odors) != 1:
            ax = ax1[j]
            ax1[0].set_ylabel("Velocity (cm/s)")
        else:
            ax = ax1
            ax.set_ylabel("Velocity (cm/s)")

        ax.set_xlabel("Time after odor onset (s)")
        ax.set_title(f"Patch {odor_label}")
        # ax.set_subtitle(f"Rig {rig}")
        ax.set_ylim(-13, max_range)
        ax.set_xlim(window)
        ax.hlines(
            5, window[0], window[1], color="black", linewidth=1, linestyles="dashed"
        )
        ax.fill_betweenx(
            np.arange(-20, max_range, 0.1),
            0,
            window[1],
            color=color_dict_label[odor_label],
            alpha=0.5,
            linewidth=0,
        )
        ax.fill_betweenx(
            np.arange(-20, max_range, 0.1),
            window[0],
            0,
            color="grey",
            alpha=0.3,
            linewidth=0,
        )

        df_results = (
            cum_trial_summary.loc[
                (cum_trial_summary.odor_label == odor_label)
                & (cum_trial_summary.is_choice == 1) 
                & (cum_trial_summary.rig == rig)
            ]
            .groupby(["odor_sites", "times", "odor_label", "mouse"])[["speed"]]
            .median()
            .reset_index()
        )

        if df_results.empty:
            continue

        if mean:
            sns.lineplot(
                x="times",
                y="speed",
                data=df_results,
                color="black",
                ci='sd',
                legend=False,
                linewidth=2,
                ax=ax,
                palette='tab20',
                hue='mouse'
            )

    sns.despine()
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Assuming cum_session is already defined and contains the data

# Identify the latest date
latest_date = cum_session['session'].max()

fig = plt.figure(figsize=(14, 14))

# Plot with jitter using swarmplot and different alpha for the latest date
for i, (y_col, title) in enumerate(zip(['distance_m', 'water_collected_ul', 'total_stops', 'session_duration_min'],
                                       ['Distance (m)', 'Water Collected (ul)', 'Total Stops', 'Session Duration (min)'])):
    ax = fig.add_subplot(2, 2, i+1)
    sns.swarmplot(data=cum_session, x='rig', y=y_col, hue='mouse', palette='tab20', ax=ax, legend=(i == 3))
    sns.swarmplot(data=cum_session[cum_session['session'] == latest_date], x='rig', y=y_col, hue='mouse', marker='X', size=8,palette='tab20',  ax=ax, alpha=1, legend=False)
    ax.set_title(title)

# Create the legend outside the loop
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles, labels, bbox_to_anchor=(1.4, 1), loc='lower right', borderaxespad=0., ncol=1)
# plt.legend(bbox_to_anchor=(1.05, 1), loc=6, borderaxespad=0., ncol=1)
sns.despine()
plt.show()

In [ ]:
for rig_name in cum_breathing.rig.unique():
    plot_breathing = cum_breathing.loc[cum_breathing.rig == rig_name]
    fig, axs = plt.subplots(2,2, figsize=(15,8), sharex=True)
    plt.suptitle(f'{rig_name}')

    for mouse, ax in zip(plot_breathing.mouse.unique(), axs.flatten()):
        stream_data = plot_breathing.loc[plot_breathing.mouse == mouse]

        ax.plot(stream_data.Time, stream_data.breathing, color= 'black')

        # ax.set_ylim(-1500, 1500)
        ax.set_xlim(1000, 1010)
        ax.set_ylabel(f'Breathing (a.u.) of {mouse}')
        ax.set_xlabel('Time (s)')

    sns.despine()
    plt.tight_layout()
    plt.show()